 ## Introduction 


<font color = "blue">
Content:

1. [Load and check Data](#1)
2. [Variable Description](#2)
3. [Data Preprocessing](#3)

1. [MODELLING](#4)
    * [Cross-Validation Hyperparameter Tuning](#5)
    * [CONCLUSION](#6)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id = "1"></a>
## Load and Check Data

In [ ]:
train = pd.read_csv('/kaggle/input/60k-stack-overflow-questions-with-quality-rate/train.csv')

In [ ]:
train.head()

<a id="2"> </a> 
## Variable Description

* Id: Unique number of each questions
* Title: The title of questions
* Body: The questions
* CreationDate: Which time question created
* Y: Quality class 
    * HQ: High-quality posts with a total of 30+ score and without a single edit. (2)
    * LQ_EDIT: Low-quality posts with a negative score, and multiple community edits. However, they still remain open after those changes. (1)
    * LQ_CLOSE: Low-quality posts that were closed by the community without a single edit. (0)


In [ ]:
train.columns

In [ ]:
train.info()

<a id = "3"></a>
## Data Preprocessing

In [ ]:
train = train[['Body','Y']]
train = train[:10000]

In [ ]:
train["Body"] = train['Body'].str.replace('<p>','')

In [ ]:
import string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.lower().replace(punctuation, '')
    return text
train["Body"] = train['Body'].apply(remove_punctuations)
train.head()

In [ ]:

train.loc[train['Y'] == 'LQ_CLOSE', 'Y'] = 0
train.loc[train['Y'] == 'LQ_EDIT', 'Y'] = 1
train.loc[train['Y'] == 'HQ', 'Y'] = 2


In [ ]:
train.head(3)

In [ ]:
train.Y = train.Y.astype('int')

In [ ]:
train.Y.value_counts()

In [ ]:
sns.countplot(x = train.Y).set_title('Category')
plt.show()

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
import nltk as nlp
nltk.download('stopwords') 
nltk.download('punkt')
nltk.download('wordnet') 

lemma = nlp.WordNetLemmatizer()

In [ ]:

text_list = []
for description in train.Body:
    
    description = re.sub('[^a-zA-Z]',' ',description)
    #description = description.lower()
    description = nltk.word_tokenize(description)
    #description = [ word for word in description if not word in set(stopwords.words('english'))]
    #lemma = nlp.WordNetLemmatizer()
    #description = [lemma.lemmatize(word)for word in description]
    description = ' '.join(description)
    text_list.append(description)

In [ ]:
len(text_list)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
max_features = 300

count_vectorizer = CountVectorizer(max_features=max_features,stop_words = "english")

space_matrix = count_vectorizer.fit_transform(text_list).toarray()  # x


In [ ]:
space_matrix.shape

<a id = '4'></a>
## MODELLING

In [ ]:
y = train.iloc[:,1].values   
x = space_matrix

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)
print('X_train shape', X_train.shape)
print('X_test shape', X_test.shape)
print('y_train shape', y_train.shape)
print('y_test shape', y_test.shape)

<a id = "5"></a>
## Cross-Validation  Hyperparameter Tuning

In [ ]:
#Libraries

from sklearn.linear_model import  LogisticRegression
from sklearn.ensemble import RandomForestRegressor , RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


In [ ]:

random_state = 42
classifier = [LogisticRegression(random_state = random_state),
                  RandomForestClassifier(random_state = random_state),
                  DecisionTreeClassifier(),
                  KNeighborsClassifier()]

logreg_param_grid = {'C': np.logspace(-3,3,7),
                    'penalty': ['l1','l2']}


rf_param_grid = rf_param_grid = {'max_features':[1,5],
                'min_samples_split': [2,3],
                'min_samples_leaf':[1,3],
                'bootstrap':[False],
                'n_estimators':[100],
                'criterion': ['gini']}

dt_param_grid = {'min_samples_split': range(10,50,2),
                'max_depth': range(1,10,2)}

knn_param_grid = {'n_neighbors': np.linspace(1,6,2, dtype = int).tolist()}

classifier_param = [logreg_param_grid ,rf_param_grid,dt_param_grid, knn_param_grid]

In [ ]:
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.metrics import accuracy_score 
cv_results = []
best_estimators = []
for i in range(len(classifier)):
    clf = GridSearchCV(classifier[i],param_grid= classifier_param[i],cv = StratifiedKFold(n_splits = 10),scoring = 'accuracy',n_jobs = -1,verbose = 1)
    clf.fit(X_train,y_train)
    cv_results.append(clf.best_score_)
    best_estimators.append(clf.best_estimator_)
    print(cv_results[i])

<a id = '6'></a>
## CONCLUSION

In [ ]:
cv_result = pd.DataFrame({'Cross Validation Means': cv_results, 'ML Models': ['LogisticRegression',
              'RandomForestClassifier',
              'DecisionTreeClassifier',
              'KNeighborsClassifier'
                ] })

g = sns.barplot('Cross Validation Means','ML Models',data = cv_result)
g.set_xlabel('Mean Accuracy')
g.set_title('Cross Validation Scores')
plt.show()

In [ ]:
cv_result

In [ ]:
best_estimators